In [1]:
import pandas as pd
import os
import torch
#import torchaudio
#import requests
#import matplotlib
#import matplotlib.pyplot as plt
#import IPython
#import gc
import numpy as np

from os.path import isfile, join
from scipy.io.wavfile import write, read
from scipy.stats import kurtosis, skew
#from datasets import load_dataset
from tqdm import tqdm
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model, Wav2Vec2ForPreTraining

In [2]:
#home_path = '/home/hpc/iwi5/iwi5055h/'
home_path = ''
aibo_path = home_path + "data/AIBO/"
save_path = aibo_path + "wav2vec/"

In [3]:
#device = 'cpu'
#if torch.cuda.is_available():
 #   device = 'cuda'

In [4]:
def Wav2Vec_Models(file, model_name='xvec', plot=False, cuda=False):

    try:
        fs, audio=read(file)   
        audio=(audio-np.mean(audio))/max(audio)
       
        device='cpu'
        if cuda:
            device='cuda'
       
        #sampling_rate = fs
       
        if model_name=='xlsr':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
            model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
        elif model_name=='base':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")   
            model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base-960h").to(device)
        elif model_name=='xvec':
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("anton-l/wav2vec2-base-superb-sv")        
            model = Wav2Vec2ForXVector.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
            #model_proj = Wav2Vec2ForPreTraining.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
   
        # audio file is decoded on the fly
        input_values = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True).to(device).input_values
        input_values_all = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True)
   
        # compute masked indices
        #batch_size, raw_sequence_length = input_values.shape
        #sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length)
        #mask_time_indices = _compute_mask_indices((batch_size, sequence_length), mask_prob=0.2, mask_length=2)
        #mask_time_indices = torch.tensor(mask_time_indices, device=input_values.device, dtype=torch.long)
       
        with torch.inference_mode():
           
            tmp = model(**input_values_all.to(device))
            #tmp_proj=model_proj(input_values.to(device), mask_time_indices=mask_time_indices.to(device) )
            if model_name=='xvec':
                last_hidden_states = tmp.embeddings.cpu()
                #last_hiddens=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))
               
                features = tmp.logits.cpu()
                #features = np.hstack((features,features_std,features_skew,features_kurt,features_min,features_max))
   
                #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))
               
               
                #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))
           
            else:
                last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())
                last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())
                last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)
                last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)
                last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())
                last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())
           
                last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))
               
                features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())
                features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())
                features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)
                features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)
                features_min=np.array(torch.min(tmp.extract_features, dim=1)[0].cpu())
                features_max=np.array(torch.max(tmp.extract_features, dim=1)[0].cpu())
               
                features = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))
   
                #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))
               
                #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))
   
        return features[0], last_hidden_states[0]
   
    except:
        print('sequence not long enough')


### Testing a file

In [6]:
test_file = 'data/AIBO/wav/Mont_01_053_00.wav'

In [7]:
model_name = 'xlsr'

In [8]:
fs, audio=read(test_file)   
audio=(audio-np.mean(audio))/max(audio)

#device='cpu'
#if cuda:
 #   device='cuda'

#sampling_rate = fs

if model_name=='xlsr':
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
    model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
    #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device)
elif model_name=='base':
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")   
    model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
    #model_proj = Wav2Vec2ForPreTraining.from_pretrained("facebook/wav2vec2-base-960h").to(device)
elif model_name=='xvec':
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("anton-l/wav2vec2-base-superb-sv")        
    model = Wav2Vec2ForXVector.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)
    #model_proj = Wav2Vec2ForPreTraining.from_pretrained("anton-l/wav2vec2-base-superb-sv").to(device)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.bias', 'project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
input_values = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True).to(device).input_values
input_values_all = feature_extractor(np.array(audio, dtype=float), sampling_rate=fs, return_tensors="pt", padding=True)
   
# compute masked indices
#batch_size, raw_sequence_length = input_values.shape
#sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length)
#mask_time_indices = _compute_mask_indices((batch_size, sequence_length), mask_prob=0.2, mask_length=2)
#mask_time_indices = torch.tensor(mask_time_indices, device=input_values.device, dtype=torch.long)

with torch.inference_mode():

    tmp = model(**input_values_all.to(device))
    #tmp_proj=model_proj(input_values.to(device), mask_time_indices=mask_time_indices.to(device) )

    last_hiddens_avg=np.array(torch.mean(tmp.last_hidden_state, dim=1).cpu())
    last_hiddens_std=np.array(torch.std(tmp.last_hidden_state, dim=1).cpu())
    last_hiddens_skew=skew(np.array(tmp.last_hidden_state.cpu()), axis=1)
    last_hiddens_kurt=kurtosis(np.array(tmp.last_hidden_state.cpu()), axis=1)
    last_hiddens_min=np.array(torch.min(tmp.last_hidden_state, dim=1)[0].cpu())
    last_hiddens_max=np.array(torch.max(tmp.last_hidden_state, dim=1)[0].cpu())

    last_hidden_states=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))

    features_avg=np.array(torch.mean(tmp.extract_features, dim=1).cpu())
    features_std=np.array(torch.std(tmp.extract_features, dim=1).cpu())
    features_skew=skew(np.array(tmp.extract_features.cpu()), axis=1)
    features_kurt=kurtosis(np.array(tmp.extract_features.cpu()), axis=1)
    features_min=np.array(torch.min(tmp.extract_features, dim=1)[0].cpu())
    features_max=np.array(torch.max(tmp.extract_features, dim=1)[0].cpu())

    features = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))

                #last_hiddens_avg=np.array(torch.mean(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_std=np.array(torch.std(tmp_proj.projected_states, dim=1).cpu())
                #last_hiddens_skew=skew(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_kurt=kurtosis(np.array(tmp_proj.projected_states.cpu()), axis=1)
                #last_hiddens_min=np.array(torch.min(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_max=np.array(torch.max(tmp_proj.projected_states, dim=1)[0].cpu())
                #last_hiddens_proj=np.hstack((last_hiddens_avg,last_hiddens_std,last_hiddens_skew,last_hiddens_kurt,last_hiddens_min,last_hiddens_max))
               
                #features_avg=np.array(torch.mean(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_std=np.array(torch.std(tmp_proj.projected_quantized_states, dim=1).cpu())
                #features_skew=skew(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_kurt=kurtosis(np.array(tmp_proj.projected_quantized_states.cpu()), axis=1)
                #features_min=np.array(torch.min(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_max=np.array(torch.max(tmp_proj.projected_quantized_states, dim=1)[0].cpu())
                #features_proj = np.hstack((features_avg,features_std,features_skew,features_kurt,features_min,features_max))
     
features[0], last_hidden_states[0]

(array([ 0.18597536, -0.5313227 , -0.501831  , ...,  1.2320832 ,
         4.237875  ,  1.8295366 ], dtype=float32),
 array([-0.03731732,  0.22137754,  0.01064559, ...,  0.21591565,
         0.01457961, -0.02043284], dtype=float32))

## Creating .csv

In [12]:
text_file = open('data/AIBO/labels/chunk_labels_5cl_corpus.txt')
file_content = text_file.read()
#print(file_content)
text_file.close()

In [13]:
##data_ = text_file.decode("utf-8").split("\r\n")
data = file_content
data = data.split("\n")
for i in range(len(data)):
      data[i] = data[i].replace('_', ' ').split(" ")
#print(data)

In [14]:
labels_df = pd.DataFrame(data, columns=["source", "speaker", "n_rec", "num", "label", "percentage"], index=None)
labels_df['filename'] = labels_df.source + "_" + labels_df.speaker + "_" + labels_df.n_rec + "_" + labels_df.num
file_path = aibo_path + "wav/"
labels_df['path'] = file_path + labels_df['filename'] + ".wav"

In [15]:
labels_df

,source,speaker,n_rec,num,label,percentage,filename,path
0,Mont,01,000,00,N,1,Mont_01_000_00,data/AIBO/wav/Mont_01_000_00.wav
1,Mont,01,001,00,N,1,Mont_01_001_00,data/AIBO/wav/Mont_01_001_00.wav
2,Mont,01,001,01,N,1,Mont_01_001_01,data/AIBO/wav/Mont_01_001_01.wav
3,Mont,01,004,00,N,0.9,Mont_01_004_00,data/AIBO/wav/Mont_01_004_00.wav
4,Mont,01,005,00,N,1,Mont_01_005_00,data/AIBO/wav/Mont_01_005_00.wav
...,...,...,...,...,...,...,...,...
18212,Ohm,32,318,00,E,0.4,Ohm_32_318_00,data/AIBO/wav/Ohm_32_318_00.wav
18213,Ohm,32,319,00,E,0.6,Ohm_32_319_00,data/AIBO/wav/Ohm_32_319_00.wav
18214,Ohm,32,320,00,N,0.9,Ohm_32_320_00,data/AIBO/wav/Ohm_32_320_00.wav
18215,Ohm,32,321,00,P,0.7,Ohm_32_321_00,data/AIBO/wav/Ohm_32_321_00.wav


In [16]:
#test_file = "data/AIBO/wav/Mont_01_007_00.wav"

In [17]:
#fs, audio = read(test_file)

In [18]:
#fs

In [19]:
labels_df["speaker"] = pd.to_numeric(labels_df["speaker"])

In [20]:
labels_df.loc[labels_df.source =='Ohm', 'speaker'] = labels_df.speaker + 25

In [21]:
labels_dict = {'A': 0, 'E': 1, 'N': 2, 'P': 3, 'R': 4}
labels_df['lb'] = labels_df['label'].map(labels_dict)

### Splitting dataset

In [22]:
labels_df.source.value_counts()

Ohm     9959
Mont    8257
           1
Name: source, dtype: int64

In [23]:
def split_indices(last=25, split_ratio=0.7, seed=13):
    np.random.seed(seed)
    train_size = int(last * split_ratio)
    val_size = int((1 - split_ratio)/2 * last)
    test_size = last - train_size - val_size
    
    a = np.arange(1, last+1, 1)
    b = np.random.permutation(a)
    chunk_size = [train_size, val_size, test_size]
    np.cumsum(chunk_size)
    c = np.split(b, np.cumsum(chunk_size))
    train, val, test = c[0], c[1], c[2]
    if len(c[3]) > 0:
        print("some samples left unused while splitting")
    return train, val, test

In [24]:
last = int(max(dict((labels_df.speaker).value_counts()).keys()))
last

57

In [25]:
train_indices, val_indices, test_indices = split_indices(last, 0.7)
split_indices(last, 0.7, 26)

(array([11, 10,  6, 37, 12, 57,  3, 23, 38, 40, 53, 43,  8, 44, 48, 42, 51,
        29, 15,  1,  9, 56, 47, 21, 50, 55, 45,  4, 41, 30, 32, 52, 34, 36,
        17, 13, 19, 28, 24]),
 array([16, 31, 22, 25, 26, 18, 46, 35]),
 array([ 5, 33, 27, 14, 20, 39,  2, 49,  7, 54]))

In [26]:
labels_df['split']=labels_df['speaker'].apply(lambda x: 'train' if x in train_indices else ('test' if x in test_indices else 'val'))


In [27]:
labels_df.to_csv(aibo_path + 'labels_df.csv')

## Extracting .npy samplewise

In [5]:
labels_df = pd.read_csv(aibo_path + 'labels_df.csv', header=0, index_col=0)

In [6]:
len(labels_df)

18217

In [7]:
labels_df

,source,speaker,n_rec,num,label,percentage,filename,path,lb,split
0,Mont,1.0,0.0,0.0,N,1.0,Mont_01_000_00,data/AIBO/wav/Mont_01_000_00.wav,2.0,train
1,Mont,1.0,1.0,0.0,N,1.0,Mont_01_001_00,data/AIBO/wav/Mont_01_001_00.wav,2.0,train
2,Mont,1.0,1.0,1.0,N,1.0,Mont_01_001_01,data/AIBO/wav/Mont_01_001_01.wav,2.0,train
3,Mont,1.0,4.0,0.0,N,0.9,Mont_01_004_00,data/AIBO/wav/Mont_01_004_00.wav,2.0,train
4,Mont,1.0,5.0,0.0,N,1.0,Mont_01_005_00,data/AIBO/wav/Mont_01_005_00.wav,2.0,train
...,...,...,...,...,...,...,...,...,...,...
18212,Ohm,57.0,318.0,0.0,E,0.4,Ohm_32_318_00,data/AIBO/wav/Ohm_32_318_00.wav,1.0,train
18213,Ohm,57.0,319.0,0.0,E,0.6,Ohm_32_319_00,data/AIBO/wav/Ohm_32_319_00.wav,1.0,train
18214,Ohm,57.0,320.0,0.0,N,0.9,Ohm_32_320_00,data/AIBO/wav/Ohm_32_320_00.wav,2.0,train
18215,Ohm,57.0,321.0,0.0,P,0.7,Ohm_32_321_00,data/AIBO/wav/Ohm_32_321_00.wav,3.0,train


In [8]:
model_mode = 'xlsr'
# 'xlsr' | 'base' | 'xvec'

In [9]:
def w2v_path(file, split):    
    return save_path + model_mode + "_extract_features/" + split + '-' + file + '.npy'

In [10]:
na_files = []
problems = []

In [ ]:
for k,v in labels_df.iterrows():
    try:
        if not isfile(v['path']):
            na_files.append(v['path'])
            print("wav for {} not available". format(v['filename']))
        elif isfile(w2v_path(v['filename'], v['split'])):
            print("w2v features found for {}".format(v['filename']))
        else:
            #ft, hs = Wav2Vec_Model(v['path'])
            ft, hs = Wav2Vec_Models(v['path'], model_name=model_mode, cuda=True)
            #np.save(save_path + "extract_features/" + v['split'] + '-' + v['filename'] + '.npy', ft)
            np.save(save_path +  model_mode + "_extract_features/" + v['split'] + '-' + v['filename'] + '.npy', ft)
            np.save(save_path +  model_mode + "_hidden_states/" + v['split'] + '-' + v['filename'] + '.npy', hs)
            print("saved")
    except:
        problems.append(v['filename'])
        print("wrong")

wav for Mont_01_000_00 not available
wav for Mont_01_001_00 not available
wav for Mont_01_001_01 not available
wav for Mont_01_004_00 not available
wav for Mont_01_005_00 not available
wav for Mont_01_007_00 not available
wav for Mont_01_008_00 not available
wav for Mont_01_009_00 not available
wav for Mont_01_010_00 not available
wav for Mont_01_011_00 not available
wav for Mont_01_012_00 not available
wav for Mont_01_013_00 not available
wav for Mont_01_014_00 not available
wav for Mont_01_015_00 not available
wav for Mont_01_016_00 not available
wav for Mont_01_016_01 not available
wav for Mont_01_016_02 not available
wav for Mont_01_017_00 not available
w2v features found for Mont_01_018_00
wav for Mont_01_020_00 not available
wav for Mont_01_021_00 not available
wav for Mont_01_021_01 not available
wav for Mont_01_022_00 not available
wav for Mont_01_022_01 not available
wav for Mont_01_023_00 not available
wav for Mont_01_024_00 not available
wav for Mont_01_025_00 not available


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['project_hid.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


saved


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['project_hid.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


saved


...

saved


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['project_hid.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


saved


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['project_hid.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


saved


## Gathering .npy arrays into train/val/test with features, hidden states and layers

In [ ]:
aibo_ft_train = []
aibo_ft_val = []
aibo_ft_test = []

In [ ]:
aibo_hs_train = []
aibo_hs_val = []
aibo_hs_test = []

In [ ]:
aibo_lb_train = []
aibo_lb_val = []
aibo_lb_test = []

In [ ]:
for file in os.listdir(save_path + model_mode + "_extract_features/"):
    if file.startswith("train-"):
        #filename = file.split("-")[1]
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        aibo_ft_train.append(ft)
    elif file.startswith("val-"):
        #filename = file.split("-")[1]
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        aibo_ft_val.append(ft)
    else:
        ft = np.load(save_path + model_mode + "_extract_features/" + file) 
        aibo_ft_test.append(ft)        
np.save(save_path + model_mode + "_aibo_ft_train.npy", aibo_ft_train)
np.save(save_path + model_mode + "_aibo_ft_val.npy", aibo_ft_val)
np.save(save_path + model_mode + "_aibo_ft_test.npy", aibo_ft_test)

In [ ]:
for file in os.listdir(save_path + model_mode + "_hidden_states/"):
    if file.startswith("train-"):
        #filename = file.split("-")[1]
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        aibo_hs_train.append(hs)
    elif file.startswith("val-"):
        #filename = file.split("-")[1]
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        aibo_hs_val.append(hs)
    else:
        hs = np.load(save_path + model_mode + "_hidden_states/" + file) 
        aibo_hs_test.append(hs)        
np.save(save_path + model_mode + "_aibo_hs_train.npy", aibo_hs_train)
np.save(save_path + model_mode + "_aibo_hs_val.npy", aibo_hs_val)
np.save(save_path + model_mode + "_aibo_hs_test.npy", aibo_hs_test)

In [ ]:
labels_df.dtypes

In [ ]:
for file in os.listdir(save_path + model_mode + "_extract_features/"):
    label = labels_df.loc[labels_df['filename'] == file.split("-")[1].split(".")[0], 'lb']
    #print(type(label))
    #print(type(int(float(label.values))))
    #print(type(str(label)))
    #lb = int(float(label.replace('\n', ' ').split(' ')[4]))
    lb = str(label)
    #lb = pd.to_string(label, index=False)
    #lb = label
    lb = int(float(lb.replace('\n', ' ').split(' ')[4]))
    if file.startswith("train-"):
        aibo_lb_train.append(lb)
    elif file.startswith("val-"):
        aibo_lb_val.append(lb)
    else:
        aibo_lb_test.append(lb)
np.save(save_path + model_mode + "_aibo_lb_train.npy", aibo_lb_train)
np.save(save_path + model_mode + "_aibo_lb_val.npy", aibo_lb_val)
np.save(save_path + model_mode + "_aibo_lb_test.npy", aibo_lb_test)

In [17]:
len(os.listdir(save_path + model_mode + "_extract_features/"))

4566

In [135]:
len(aibo_lb_val)

670

In [19]:
len(aibo_lb_train)

3269

In [136]:
len(aibo_hs_val)

670

In [20]:
len(aibo_lb_test)

627

In [130]:
len(aibo_hs_test)

627

In [134]:
aibo_lb_train[634]

1

In [103]:
type(aibo_lb_test[0].replace)

str

In [131]:
len(aibo_lb_val)

670

In [118]:
#str(aibo_lb_val[0]).replace('\n', ' ').split(' ')

['2']

In [137]:
#for i in range(len(aibo_lb_val)):
 #   a = str(aibo_lb_val[i]).replace('\n', ' ').split(' ')[4]
  #  aibo_lb_val[i] = int(float(a))

In [138]:
type(int(float(aibo_lb_test[0])))

int

In [31]:
len(aibo_lb_train[2].split(' '))

8

In [40]:
int(float(aibo_lb_train[2].replace('\n', ' ').split(' ')[4]))

1

In [39]:
type(int(float(aibo_lb_train[2].replace('\n', ' ').split(' ')[4])))

int

In [ ]:
#np.unique(labels)

In [17]:
#len(os.listdir('data/AIBO/wav/'))

3547

In [21]:
len(na_files)

14874